In [1]:
text = """123
4567
 8910
"""
with open('task_1.1_source', 'w') as f:
    f.write(text)

In [2]:
!type task_1.1_source

123
4567
 8910


# Базовый уровень
## Задание 1.1

Напишите функцию, копирующую файл

source_path должен существовать, а отсутствующие промежуточные папки из target_path должны быть созданы

Подсказка: можно прочитать исходный файл и записать в новый.

Опциональное усложнение: попробуйте написать так, чтобы функция работала и с большими файлами (нужно, чтобы содежимое файла не считывалось полностью в память)


In [4]:
import os
import re
def copy_file(source_path, target_path):
    if os.path.isfile(source_path): # проверяем наличие файла source_path
        target_dirs = target_path
        target_path_splitted = re.split('["\","/"]', target_path) # разобъем на составляющие target_path, все время путаю слеши
        if len(target_path_splitted) > 1: # если target_path состоит не только из имени, то
            target_dirs = target_path_splitted[0] # создадим накопительную переменную из промежуточных папок
            if not os.path.exists(target_dirs): # проверяем существование первой папки из target_path в текущей папке
                os.makedirs(target_dirs) # создадиим ее, если ее нет.
            for index in range(1, len(target_path_splitted)-1): # далее пройдем глубже по всем папкам target_path
                target_dirs = os.path.join(target_dirs, target_path_splitted[index]) # с корректным системным разделителем папок
                if not os.path.exists(target_dirs): # и если какая-либо из них не существует - 
                    os.makedirs(target_dirs) # создаем ее
            # на этот момент все папки из target_path существуют
            target_dirs = os.path.join(target_dirs,target_path_splitted[-1]) # добавим имя файла в конец этой переменной
        with open(source_path, 'r') as f1, open(target_dirs, 'w') as f2: # откроем файлы для чтения/записи
            for line in f1: # данная конструкция аналогична методу xreadlines,        
                f2.write(line) # который считывает файл строка за строкой, а не весь сразу, экономя тем самым память
    else: return "source file don't exists"
    
copy_file('task_1.1_source', 'task_1.1_target')
!type task_1.1_target

123
4567
 8910


# Задание 1.2
В файле source_path находится англо-латинский словарь, то есть список слов на английском языке и их переводы на латинский язык (переводов может быть несколько). Необходимо создать из него латино-английский словарь по пути target_path. Например, из словаря

apple - malum, pomum, popula

fruit - baca, bacca, popum

punishment - malum, multa
нужно сделать словарь.

baca - fruit

bacca - fruit

malum - apple, punishment

multa - punishment

pomum - apple

popula - apple

popum - fruit
Не забудьте, что слова нужно расположить в алфавитном порядке.
Будет полезна функция strip у строки для удаления пробелов

In [92]:
import os, re
def eng_lat_2_lat_eng(source_path, target_path):
    with open(source_path, 'r') as f:
        dict1 = f.read() # Сперва весь файл поместим в память
    dict1 = dict1.split('\n') # далее разобьем его на строки
    my_dict = dict() # тут будет храниться финальный словарь
    my_keys = [] # а тут его уникальные ключи, для удобства их сортировки
    for line in dict1:
        line_splitted = line.split('-') # отделим ключи от значений
        keys = line_splitted[1].split(',') # разделим значения друг от друга, они станут ключами в итоговом словаре
        value = line_splitted[0] # сюда будем помещать ключи предыдущего словаря, они станут значениями.
        for i in keys:
            val_temp = '' # эта переменная нужна для пополнения словаря, без нее происходит затирание предыдущих значений
            key = i.strip(' ') # уберем лишние пробелы...
            if not key in my_dict: my_keys.append(key) # если ключа еще нет в нашем словаре - он уникален, добавим в список
            if key in my_dict: 
                val_temp = my_dict.get(key) + ", " # для уже существующих ключей добавим значение к имеющемуся. 
            my_dict[key] = val_temp + value.strip(' ') # наполним словарь
    my_keys = sorted(my_keys) # отсортируем ключи для вывода в файл
    with open(target_path, 'w') as f:
        for keys in my_keys:
            line = keys + " - " + my_dict[keys] + "\n"
            f.write(line) # формируя каждую строку, запишем файл. 
    return

eng_lat_2_lat_eng('task_1.2_source', 'task_1.2_target')
!type task_1.2_target

baca - fruit
bacca - fruit
malum - apple, punishment
multa - punishment
pomum - apple
popula - apple
popum - fruit


    
# Задание 1.3

Напишите функцию, вычисляющую длину русского слова, независимо от его кодировки

Если параметр encoding задан, то нужно декодировать текст при помощи этой кодировки. А если он не задан, то кодировку надо определить самостоятельно.
Если кодировка не задана, то точное решение здесь написать не получится. Но можно воспользоваться функцией my_decoder из лекции


In [122]:
import chardet
def my_len(word, encoding=None):
    if encoding:
        return len(word.decode(encoding)) 
    if type(word) is unicode:
        return len(word)
    else: return len(unicode(word, encoding=chardet.detect(word)['encoding']))


In [124]:
my_len('текст') == 5

True

In [123]:
my_len(u'текст') == 5

True

In [125]:
my_len(u'ааааа'.encode('utf_16'), encoding='utf_16') == 5

True

In [126]:
my_len(u'текст'.encode('windows-1251')) == 5

True


## Продвинутый уровень
# Задание 2.1

Напишите функцию, которая выписывает список файлов в данной директории и сортирует их в соответствии с их размером. Функция должна получать путь к директории в качестве аргумента и печатать на экран имена всех файлов в ней и их размеры, причем первыми должны идти файлы с наибольшими размерами, а в случае одинакового размер файлы сортируются по алфавиту.
Указание. Изучите функции listdir и stat из модуля os и функции isfile и join из модуля os.path.

In [ ]:
def get_dir_content_info(dir_path):
    # Допишите код
    return
